In [7]:
import re
from IPython.display import clear_output

In [8]:
# define rooms and items

couch = {
    "name": "couch",
    "type": "furniture",
}

door_a = {
    "name": "door a",
    "type": "door",
}

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

piano = {
    "name": "piano",
    "type": "furniture",
}

room_a = {
    "name": "game room",
    "type": "room",
}

room_b = {
    "name": "bedroom 1",
    "type": "room",
}

queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

room_c = {
    "name": "bedroom 2",
    "type": "room",
}

double_bed = {
    "name": "double bed",
    "type": "furniture",
}

dresser = {
    "name": "dresser",
    "type": "furniture",
}


key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}

room_d = {
    "name": "living room",
    "type": "room",
}

dining_table = {
    "name": "dining table",
    "type": "furniture",
}

door_d = {
    "name": "door d",
    "type": "door",
}

key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}

secret_door = {
    "name": "secret door",
    "type": "door",
}

secret_key = {
    "name": "secret key",
    "type": "key",
    "target": secret_door,
}

outside = {
  "name": "outside"
}

all_rooms = [room_a, room_b, room_c, room_d, outside]

all_doors = [door_a, door_b, door_c, door_d, secret_door]

# define which items/rooms are related

object_relations = {
    #GAME ROOM RELATIONS
    "game room": [couch, piano, door_a, secret_door],
    "piano": [key_a],
    "door a": [room_a, room_b],
    #ROOM B RELATIONS
    "bedroom 1": [queen_bed, door_b, door_a, door_c],
    "queen bed": [key_b],
    "door b": [room_b, room_c],
    "door c": [room_b, room_d],
    #ROOM C RELATIONS
    "bedroom 2": [double_bed, dresser, door_b],
    "double bed": [key_c],
    "dresser": [key_d],     
    #ROOM D RELATIONS
    "living room": [dining_table, door_c, door_d],
    "door d": [room_d, outside],
    #OUTSIDE
    "outside": [door_d],
    "secret door": [room_a, outside]
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": room_a,
    "keys_collected": [secret_key],
    "target_room": outside,
    "movements": 0,
}

game_state = {
    "current_room": room_a,
    "keys_collected": [secret_key],
    "target_room": outside,
    "movements": 0,
}

In [9]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def no_signs_and_lower(string):         #Function to convert user inputs to lower case and take out the signs
    string = string.lower()
    pattern = "\W^\s"
    string = re.sub(pattern, "", string)
    pattern = "\n"    
    string = re.sub(pattern, "", string)
    return string

def start_game():
    """
    Start the game
    """
    print("""You wake up on a couch and find yourself in a strange house with no windows which you
have never been to before. \U0001F635 You don't remember why you are here and what had happened
before. You feel some unknown danger is approaching and you must get out of the house, NOW!""")
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        clear_output()
        print("Congrats! \U0001F973 You escaped the room!")
        movs = ""
        if game_state["movements"] == 1:     #Print signular or plural for movements and the number of movements
            movs = "movement"
        else:
            movs = "movements"
        print(f'You did it in {game_state["movements"]} {movs}.')        
        if input("Do you want to play again? Enter 'yes' or 'no'").strip() == 'yes':   #Option to play again
            clear_output()
            linebreak()
            play() 
        else:
            print("Ohhhh! Bye Bye then. \U0001F44B")  # End if you do not want to play again
    else:
        print("\nYou are now in " + room["name"] + ".")     
        intended_action = input("What would you like to do? Type 'explore' or 'examine <item>'.").strip()
        intended_action = no_signs_and_lower(intended_action)
        if intended_action == "explore":
            game_state["movements"] += 1
            explore_room(room)
            play_room(room)
        elif intended_action.split()[0] == "examine":  # if the input start with examine, look for the item
            game_state["movements"] += 1  
            if re.search("examine \w+", intended_action) is not None:
                pattern = "\w+"
                list_item =  re.findall(pattern, intended_action)
                item = ' '.join(list_item)              
                item = re.sub("examine ", "", item)
                examine_item(no_signs_and_lower(item))                
            else:
                item = input("What would you like to examine?").strip()
                examine_item(no_signs_and_lower(item))
        else:
            print(f"You typed {intended_action}. Not sure what you mean. Type 'explore' or 'examine <item>'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.   
    """
    items = [i["name"] for i in object_relations[room["name"]] if i["name"] != "secret door"] #Don't print the secret door
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))
              
def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    clear_output()
                    output += "You unlock it with a key \U0001F5DD you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + " \U0001F5DD ."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")
    
    if next_room:
        linebreak()
        play_room(next_room)
    else:
        play_room(current_room)

def play():                                                #Initialize the game_state and start the game
    game_state["current_room"] = room_a
    game_state["keys_collected"] = [secret_key]
    game_state["target_room"] = outside
    game_state["movements"] = 0
    start_game()              

In [10]:
#Run all the cells above and then this cell to play Escape Room
play()

Congrats! 🥳 You escaped the room!
You did it in 1 movement.
Do you want to play again? Enter 'yes' or 'no'no
Ohhhh! Bye Bye then. 👋





### IMPROVEMENTS:
- Changes the input to **lowercase and takes out the symbols** (except the space). Function **no_signs_and_lower**: .
- **Do not ask permision** to go to the next room if you have the correct key.
- Put a **secret door** in game room to the outside (**Easter Egg**). Input "Examine secret door" in game room.
- Count the **number of movements** the player did before exiting outside (if is just one, write movements in singular). Var **movements**.
- **Clear output** line, every time you change a room 
- Add the possibility to **play again**.
- Start with a single line. Function **play()**
- Add the possibility to **enter examine + item** directly in one step.
- Add a few **emojis** in the text with Unicode.